In [26]:
import sys
sys.path.append("../")
from data_retrieval.data import split_data
from torchvision import transforms
from torchvision.transforms import Resize
from torchvision.models import resnet50
import torch
import json
import torch
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
transformations = Resize((256,256))
mean, std = split_data.get_mean_std('../data_retrieval/data/annotations.csv', '../data_retrieval/data/', transformations)

e:\Anaconda\envs\Babylon\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [10]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=mean, std=std)
])

In [12]:
train, val, test = split_data.split_data('../data_retrieval/data/annotations.csv', '../data_retrieval/data/', data_transforms)

In [13]:
model = resnet50()

In [14]:
model.load_state_dict(torch.load('../models/models_weights/resnet50.pth'))

<All keys matched successfully>

In [15]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
test_loader = torch.utils.data.DataLoader(test,
                                             batch_size=16, shuffle=True,
                                             num_workers=4)

In [19]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

In [22]:
f = open("../data_retrieval/data/mapping")
mapping = json.load(f)

In [24]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in mapping.values()}
total_pred = {classname: 0 for classname in mapping.values()}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[mapping[str(int(label))]] += 1
            total_pred[mapping[str(int(label))]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: E_Neo-Assyrian is 75.0 %
Accuracy for class: E_Neo-Babylonian is 88.6 %
Accuracy for class: GAR_Neo-Assyrian is 78.6 %
Accuracy for class: GAR_Neo-Babylonian is 91.0 %
Accuracy for class: KA_Neo-Assyrian is 82.6 %
Accuracy for class: KA_Neo-Babylonian is 83.7 %
Accuracy for class: KI_Neo-Assyrian is 63.4 %
Accuracy for class: KI_Neo-Babylonian is 86.4 %
Accuracy for class: MEŠ_Neo-Assyrian is 75.9 %
Accuracy for class: MEŠ_Neo-Babylonian is 84.0 %
Accuracy for class: NI_Neo-Assyrian is 79.4 %
Accuracy for class: NI_Neo-Babylonian is 73.9 %
Accuracy for class: RU_Neo-Assyrian is 63.6 %
Accuracy for class: RU_Neo-Babylonian is 84.2 %
Accuracy for class: TA_Neo-Assyrian is 77.8 %
Accuracy for class: TA_Neo-Babylonian is 90.9 %
Accuracy for class: TI_Neo-Assyrian is 58.3 %
Accuracy for class: TI_Neo-Babylonian is 94.4 %
Accuracy for class: U₂_Neo-Assyrian is 85.7 %
Accuracy for class: U₂_Neo-Babylonian is 74.5 %
Accuracy for class: ŠU_Neo-Assyrian is 78.3 %
Accuracy for

In [ ]:
y_pred = []
y_true = []

# iterate over test data
for inputs, labels in test_loader:
        output = model(inputs) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)